#14757 Homework 1  (150 pts)

## **Due:** Wednesday September 18 at 3pm ET / 12 noon PT

## Submission Instructions

*   Download your completed notebook by clicking File->Download .ipynb and submit it on Gradescope
*   Check your submission on Gradescope to make sure that all your code, code output and written responses appear correctly

## Problem 1: Gradescope Autograder Placeholder (0 pts)

Gradescope requires that problem 1 be autograded for code submissions, but there are no autograded problems. Continue to problem 2.

## Problem 2: The Wrongful Conviction of Sally Clark (20 pts)

Sally Clark was a British lawyer who was wrongly sentenced to life in prison in 1999 for the deaths of her two infant children. Her elder son Christopher died at age 11 weeks in December 1996 and her younger son Harry at 8 weeks in January 1998. At her trial, the defence argued that the deaths were due to sudden infant death syndrome (SIDS). Clark was convicted on the basis of testimony by pediatrician Sir Roy Meadow, who made the following argument:

*   Hospital records show that the ratio of SIDS deaths to live births in affluent non-smoking families is about $\frac{1}{8500}$. (A live birth is a birth in which a child is born alive; not a still birth.)
*   The chance of two SIDS deaths occurring in the same family is about $\frac{1}{8500^2} \approx \frac{1}{73000000}$.
*   It is therefore extremely unlikely that Clark is innocent.

As a result of this prosecution, Clark spent more than 3 years in prison and was finally exonerated in 2003 after it was determined that Meadow’s expert testimony was flawed. Two other women against whom Meadow provided expert testimony had their convictions overturned as well.



**2.1** (10 pts) Identify a flaw in Meadow’s $\frac{1}{73000000}$ figure.

WRITE YOUR ANSWER HERE:
The problem with Meadow's figure is that he assumes conditional independence meaning that the 2 infant deaths are uncorrelated such that the probability of 2x SIDS deaths in the same family is 1/8500. However, Meadows being a pediatrician should recognize that 2 children being afflicted with the same disease coming from the same family should not be asssumed as conditionally independent or at least investigate and prove that assumption. Therefore his assumption would fail thus rendering his figure completely incorrect.

**2.2** (10 pts) Even if we accept Meadow’s $\frac{1}{73000000}$ calculation as valid, what is wrong with a juror interpreting it as the probability of Clark’s innocence?

WRITE YOUR ANSWER HERE:
Even if we accept Meadow's Number as valid, the juror should not be concerned with Meadow's Number as his calculation gives the probability of 2x SIDS deaths given Clark's innocence P(2x SIDS Deaths | Clark's Innocence). First and foremost, the juror should be concerned with the probability that Clark is innocent given the two natural infant deaths regardless of cause. It stands to reason that $P(2x SIDS Deaths | Clark's Innocence) != P(Innocence | 2x Natural Infant Deaths)$


## Problem 3: Attacking a Multinomial Naive Bayes Spam Filter (80 pts)



In this problem you will execute several attacks on a naive Bayes spam filter for text messages (SMS). Run the cell below to set up the spam filter. The code performs the following steps:

*   Reading and displaying the data
*   Splitting the data into training and testing sets
*   Converting each text message into a vector of word counts, so that the training messages are represented as a single matrix, and so are the test messages
*   Training the spam filter using the training set and evaluating it with respect to the testing set

Helpful documentation:

*   https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
*   https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

This implementation is adapted from source code here: https://github.com/rahulvasaikar/SMS-spam-Detection

The original data is documented here: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [2]:
# READING AND DISPLAYING DATA

# Reading tab separated text file into a Pandas dataframe
import pandas as pd
url = 'http://www.andrew.cmu.edu/user/dvaroday/14757/data/hw1/SMSSpamCollection'
df = pd.read_table(url, sep='\t', names=['label', 'message'])

# Mapping ham/spam labels to 0/1 by creating a new column and using the .map() method to determine the matching map values
df['binary_label'] = df.label.map({'ham':0,'spam':1})

# Display dataframe
print('DATASET')
display(df)

# SPLITTING INTO TRAINING AND TESTING SETS

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'],
                                        df['binary_label'], random_state=1)

# 2D dataframe shape prints in (num_rows, num_columns)
total_rows = df.shape[0]
training_rows = X_train.shape[0]
test_rows = X_test.shape[0]

# I would like to see a percent here, but don't wanna mess it up
print('\nDATA SPLIT STATISTICS')
print('Number of rows in the total set:    {}'.format(total_rows))
print('Number of rows in the training set: {}'.format(training_rows))
print('Number of rows in the test set:     {}'.format(test_rows))

# CONVERTING EACH TEXT MESSAGE INTO A VECTOR OF WORD COUNTS
'''Find out if this is TF-IDF or likely just straight word counts'''

from sklearn.feature_extraction.text import CountVectorizer
# Instantiate the CountVectorizer
count_vector = CountVectorizer()

'''
This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.

If you do not provide an a-priori dictionary and you do not use an analyzer that does 
some kind of feature selection then the number of features will be equal to the vocabulary
size found by analyzing the data.

The "feature counts" are just be the words and values are associated
amount of times that word occurs for a given label'''

# Fit the training data
# Returns matrix of feature counts in training text messages
# Features are obtained from the training set itself
training_data = count_vector.fit_transform(X_train)


'''
Why is fit_transform used for training data, but transform is used for testing data?

.fit() - Learn a vocabulary dictionary of all tokens in the raw documents.

.transform() - produces the document-term matrix

So fit presumably applies some form of separate transformation on the feature vectors

Take to different notebook and play with it
'''

# Transform testing data
# Returns matrix of feature counts in test text messages
# Note that the features are the ones obtained from the training set
testing_data = count_vector.transform(X_test)
'''Find out how it transforms the data, but should be labeled somewhere
Example document-term matrix
                Feature Vectors (aka words)
Docs        Hey     Bro     Send    Money
Doc-1       1       0       3           3
Doc-2        1      2       1           0

'''


# TRAINING AND EVALUATING MODEL

# Train
from sklearn.naive_bayes import MultinomialNB
# MultinomialNB with Laplacian smoothing (alpha=1.0)
spam_filter = MultinomialNB(alpha=1.0)
spam_filter.fit(training_data, y_train)
'''
Basically creating the Naive Bayes classifier function with .fit()
with both training_data labels and the ham/spam labels. 



Need to break this down later, but find out what the data looks like
after y_train labels are applied to document-term matrix
'''

# Evaluate
predictions = spam_filter.predict(testing_data)
from sklearn.metrics import accuracy_score, recall_score
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions)

print('\nPERFORMANCE')
print('Accuracy:', "{:.3f}".format(accuracy))
print('Recall:  ', "{:.3f}".format(recall))

DATASET


,label,message,binary_label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will ü b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0



DATA SPLIT STATISTICS
Number of rows in the total set:    5572
Number of rows in the training set: 4179
Number of rows in the test set:     1393

PERFORMANCE
Accuracy: 0.989
Recall:   0.941


### Understanding baseline performance

**3.1** (10 pts) In the cell above *accuracy* is the fraction of messages that are correctly classified by the spam filter. *Recall* is the fraction of true spam messages correctly classified as spam. But the message recipient cares about true ham messages that are classified as spam because these messages are filtered out by the spam filter. Neither accuracy nor recall address this issue directly. Instead, the metric *specificity* represents the fraction of true ham messages that the recipient will actually receive. Fix the code between the comments in the cell below to calculate specificity for the test set.

https://en.wikipedia.org/wiki/Precision_and_recall

https://en.wikipedia.org/wiki/Sensitivity_and_specificity

In [3]:
def specificity_score(y_test, predictions):
  test_messages = len(y_test.values)
  true_spams = sum(y_test.values)
  positives = sum(predictions)
  true_positives = sum(y_test.values * predictions)

  # START EDITING HERE - DON'T REMOVE THIS COMMENT
  '''
  Sensitivity = TP/(TP + FN) aka probability of a positive test result GIVEN actual positive
  Specificity = TN / (TN + FP) aka probabililty of a negative test result GIVEN actual negative

  Think COVID

  This is pretty weird though because even the referenced wiki link says true negative rate
  But we don't have negatives here so, equation really isn't that different anyway

  y_test from above is the actual values of the test data
  predictions is the output from running the test data throught the classifier
  predictions = spam_filter.predict(testing_data)

  since Ham is 0 and Spam is 1, simple sum here suffices
  true_spams = all actual spam messages
  positives = all predicted positives positives
  true_positives performs vector multiplication then sums because 1*1 = 1 and anything*0=0
  so sum still works

  Also, true positives is the spam classifier working. True Negatives is the number of messages
  correctly classified as ham
  '''
  true_hams = test_messages - true_spams
  # probably don't even need false positives plus true negatives values with true_hams values, but makes me feel better to do the full equation
  negatives = len(predictions) - positives
  false_positives = positives - true_positives 
  true_negatives = true_hams - false_positives
  spec = true_negatives / (true_negatives + false_positives)

  return spec
  # END EDITING HERE - DON'T REMOVE THIS COMMENT

specificity = specificity_score(y_test, predictions)
print('Specificity:', "{:.3f}".format(specificity))

Specificity: 0.996


### Evasion attack

**3.2** (10 pts) In this part you act as a spammer who wants to evades the spam filter. Edit `evasion_message` in the cell below so that its probability of being labeled spam becomes less than 1%. For full credit, `evasion_message` must convey the same meaning as `original_message`.

In [4]:
original_message = 'Txt back to get your $1000 prize now'

# EDIT THE MESSAGE BELOW - DON'T REMOVE THIS COMMENT
evasion_message = 'Text me back now to receive the contest prize money that you are owed in the amount of one thousand dollars'

# Probabilistic predictions of original_message and evasion_message
df_evasion = pd.DataFrame([original_message, evasion_message],
                          columns=['message'])
message_data = count_vector.transform(df_evasion['message'])
prediction_probabilities = spam_filter.predict_proba(message_data)

print('Probability that original message is spam: ',
      "{:.6f}".format(prediction_probabilities[0,1]))
print('Probability that evasion message is spam:  ',
      "{:.6f}".format(prediction_probabilities[1,1]))

Probability that original message is spam:  0.999985
Probability that evasion message is spam:   0.003253


**3.3** (10 pts) Describe the process you used to create your `evasion_message` in 1-2 sentences. Was this an open box (whitebox) or closed box (blackbox) attack? If open box, specify what information about `spam_filter` you used.

WRITE YOUR ANSWER HERE: While we technically have access to the model, you can see that in the above cells that no data or specific weights were queried, and my approach to beating the spam filter was iterative despite one line evasion message. Therefore, this was a closed box attack as I didn't adjust my message based on prior knowledge of the decision boundary.

### Poisoning attack

**3.4** (10 pts) In this part you play the role of an adversary working against a ride share company. The company wants to deliver `target_message` to its customers. You gain access to the training set and have the opportunity to poison the training data with one poison message/label pair, but your example may be rejected by the spam filter administrator. Edit `poison_message` and/or `poison_label` so that they are accepted by the administrator and make the probability of `target_message` being labeled spam greater than 50%.

In [5]:
target_message = 'Your ride share has arrived.'
df_target = pd.DataFrame([target_message], columns=['message'])
target_data = count_vector.transform(df_target['message'])
target_probabilities = spam_filter.predict_proba(target_data)
print('Probability that target message is spam before poisoning:',
      "{:.6f}".format(target_probabilities[0,1]))

# START EDITING HERE - DON'T REMOVE THIS COMMENT
poison_message = 'Your ride share has arrived. Get your $1000 prize from the ride share'
poison_label = 1
# END EDITING HERE - DON'T REMOVE THIS COMMENT

# Administrator runs poison message through original spam filter
# to validate its label
df_poison = pd.DataFrame([[poison_message, poison_label]],
                         columns=['message', 'binary_label'])
poison_data = count_vector.transform(df_poison['message'])
poison_validation = spam_filter.predict(poison_data)

if poison_label != poison_validation[0]:

  print('Administrator rejects your poison message/label')

else:

  print('Administrator accepts your poison message/label')

  # Initialize retraining data with original training data and append poison
  df_retrain = pd.DataFrame(X_train, columns=['message'])
  df_retrain['binary_label'] = y_train.values
  df_retrain = pd.concat([df_retrain, df_poison], ignore_index=True)

  # Retrain
  recount_vector = CountVectorizer()
  retraining_data = recount_vector.fit_transform(df_retrain['message'])
  retrained_filter = MultinomialNB(alpha=1.0)
  retrained_filter.fit(retraining_data, df_retrain['binary_label'])

  # Evaluate target message with retrained_filter
  recount_target_data = recount_vector.transform(df_target['message'])
  poisoned_probabilities = retrained_filter.predict_proba(recount_target_data)

  print('Probability that target message is spam after poisoning: ',
        "{:.6f}".format(poisoned_probabilities[0,1]))

Probability that target message is spam before poisoning: 0.037813
Administrator accepts your poison message/label
Probability that target message is spam after poisoning:  0.661127


**3.5** (10 pts) Describe an additional way for the administrator to decide whether to reject the poison message/label pair in order to defend against this type of attack. Use 1-4 sentences in your answer.

WRITE YOUR ANSWER HERE:
Assuming we are monitoring our classifier, we could put incoming messages through a less sophisticated filtering method to catch possible anomalies. We could compare the incoming messages' count vectors against the training data count vectors as a form of anomaly detection. Example: If you were receiving my potential spam message, you could compare it to other spam messages and see the $1000 token count of those messages as a clear indicator of spam in the training set and compare the rest of the count vectors. It's likely that there won't be many spam messages from the training set with similar 'ride' or 'share' counts, so it's an anomaly.

### Reverse engineering attack

**3.6** (20 pts) In this part you will perform a reverse engineering attack against the spam filter administrator. Your goal is to reconstruct the `spam_filter` model without using any of the original training messages or labels. In the starter code below the attacker sends a set of `probe_messages` to the `spam_filter`. The attacker can detect whether each message has been classified as spam or ham by inspecting its read receipt. The attacker uses these classifications as labels to train `reconstructed_filter`.

Edit `probe_messages` so that `reconstructed_filter` achieves at least 97% accuracy and 88% recall on the test set. You can use at most `max_probes = 5000` probe messages. You may **not** use the training or test sets to help construct `probe_messages`, **except** for the list of words provided to you in the list `vocab`.

In [6]:
import numpy

max_probes = 5000
vocab = count_vector.get_feature_names_out()


# START EDITING HERE - DON'T REMOVE THIS COMMENT
# probe_messages = ['probe ' + str(i) for i in range(max_probes)] commented so I don't lose it
'''
I think it would be best to do this recursively, with getting probe messages through the filter
for labels then data analysis on that to find more, organize the data by labels, then readjust
from there, but I'm not sure if we're allowed to do that. 
'''
# probe_messages = [str(vocab[i+1700]) for i in range(max_probes)] # simply using the probe messages from the vocab list gave me Accuracy: 0.952 and Recall: 0.876
# print(probe_messages)

'''
Probe Table
i:      Accuracy: 0.952 and Recall: 0.876
i+500:  Accuracy: 0.958 and Recall: 0.865
i+1000: Accuracy: 0.95 and Recall: 0.865 
i+1500: Accuracy: 0.945 and Recall: 0.908 
i+1700: Accuracy: 0.972 and Recall: 0.870
i+2000: Accuracy: 0.966 and Recall: 0.843 
i+2300: Accuracy: 0.976 and Recall: 0.838
'''

# probe_messages = [str(vocab[i+1500]) for i in range(max_probes)]
# probe_messages = [str(vocab[i+1700]) for i in range(max_probes)]
# # probe_messages = probe_messages1[1000:3500] + probe_messages2[:1200] + probe_messages2[3500:3800] # Accuracy: 0.933 and Recall: 0.87, but not maximizing labels
# probe_messages = probes[:500] + probes[1000:3500] + + probes[3500:4800] # Accuracy: 0.926 and Recall: 0.903, but not maximizing labels


'''
abandoned binning approach because it simply lacked any form of control

Couldn't figure out how to do it without ensuring that I had as many spam messages
as possible which requires going through the classifier for at least one go round
to figure out all actual spam messages and then going from there.
'''
all_probes = [str(vocab[i]) for i in range(len(vocab))]
df_all_probes = pd.DataFrame(all_probes, columns=['message'])
probe_all_data = count_vector.transform(df_all_probes['message'])
probe_all_labels = spam_filter.predict(probe_all_data)
# print(probe_all_labels)
df_all_probes['label'] = probe_all_labels
# print(len(spam_messages)) #783
'''
Took all of the spam convert to list and then take all of the ham, convert to list,
then only include max_probes minus amount of spam for total probe_messages within max_probe count
'''
spam_messages = df_all_probes[df_all_probes['label'] == 1]['message'].tolist()
ham_messages = df_all_probes[df_all_probes['label'] == 0]['message'].tolist()
ham_messages = ham_messages[::-1]
spam_messages_2_word = [spam_messages[i] + ' ' + ham_messages[i] for i, j in enumerate(spam_messages) if len(spam_messages) > 2]

probe_messages = spam_messages + spam_messages_2_word + ham_messages[:(max_probes - len(spam_messages) - len(spam_messages_2_word))]

# END EDITING HERE - DON'T REMOVE THIS COMMENT

# Run probe_messages through spam_filter to obtain probe_labels
'''
Put probes into a dataframe send that dataframe through the transform() method to
generate document-term matrix
Then, create labels for the messages by running the probe data through the classifier
'''
print('Number of probe messages:', len(probe_messages))

df_probes = pd.DataFrame(probe_messages, columns=['message'])
probe_data = count_vector.transform(df_probes['message'])
probe_labels = spam_filter.predict(probe_data)

# Train reconstructed_filter using probe_messages and probe_labels
'''
Having run the classifier to create labels, we then create a new classifier
using the probes and labels already created, then find their predictions
'''
reconstructed_filter = MultinomialNB(alpha=1.0)
reconstructed_filter.fit(probe_data, probe_labels)
reconstructed_predictions = reconstructed_filter.predict(testing_data)

# Evaluate
accuracy = accuracy_score(y_test, reconstructed_predictions)
recall = recall_score(y_test, reconstructed_predictions)

print('Accuracy:', "{:.3f}".format(accuracy))
print('Recall:  ', "{:.3f}".format(recall))

Number of probe messages: 5000
Accuracy: 0.970
Recall:   0.903


**3.7** (10 pts) Describe one way that the administrator can defend against this type of attack. Use 1-4 sentences in your answer.

WRITE YOUR ANSWER HERE:
Honestly, I don't think there are many algorithmic ways to harden the classifer against this form of brute force attack other than simply limiting the number of messages that can pass through the administrator at a given time. So rate-limiting requests if there's an API as well.

## Problem 4: Implementing a Multinomial Naive Bayes Classifier (50 pts)

In this problem you will implement a Multinomial Naive Bayes classifier from scratch that performs identically to the one from the `sklearn.naive_bayes` library you used above. First run the cell below to load the preprocess the data.

In [7]:
# Reading tab separated text file into a Pandas dataframe
import pandas as pd
url = 'http://www.andrew.cmu.edu/user/dvaroday/14757/data/hw1/SMSSpamCollection'
df = pd.read_table(url, sep='\t', names=['label', 'message'])

# Mapping ham/spam labels to 0/1
df['binary_label'] = df.label.map({'ham':0,'spam':1})

# Splitting into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'],
                                        df['binary_label'], random_state=1)

# Converting each text message into a vector of word counts
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train)
testing_data = count_vector.transform(X_test)

In the cell below complete the class `MultinomialNB` by implementing the `fit()` and `predict()` methods. The constructor `__init__()` is written for you but you can modify it according to your needs.

For full credit your implementation should not use any loops. Instead you should operate directly on vectors and matrices, not iterate through them. In particular your `fit()` method should calculate log-probability ratios with `np.log()` so that your `predict()` method can use `np.matmul()` for matrix multiplication. For your convenience the starter code already converts the method inputs into `numpy` arrays. Here's a guide to vectorization and broadcasting with `numpy`: https://realpython.com/numpy-array-programming

Make the following assumptions:

*   The classification task is binary. Specifically `train_labels` is a vector of values 0 and 1 only.
*   The feature values in `train_features` and `test_features` are nonnegative integers.

In [14]:
import numpy as np

# START EDITING HERE - DON'T REMOVE THIS COMMENT

class MultinomialNB:

  def __init__(self, alpha):
    self.alpha = alpha # Equal to 1.0 for Laplace smoothing
    self.label_log_prob_ratios = None
    self.feature_log_prob_ratios = None
    self.label_counts = None

  def fit(self, train_features, train_labels):
    '''
    train_features is the document-term sparse matrix and convertd to 2D numpy array (num_messages, num_features)
    '''
    train_features = np.array(train_features.todense())  # todense() converts the sparse matrix to the dense matrix
    train_labels = np.array(train_labels)  # 1D array of spam:1 and ham:0, pretty simple
    
    '''
    MultinomialNB is not always a binary classifier so here,
    we could hardcode 0,1 here only, but that's limits the overall class
    '''
    # Calculate ratios
    '''
    label_counts creates a numpy array by finding the counts for each label in train_labels
    and np.unique is a quick and robust way to get all the possible labels,
    then list comprehension to find their sums aka counts
    '''
    num_labels = np.unique(train_labels)
    label_counts = np.array([(train_labels == label).sum() for label in num_labels])
    n_samples, n_features = train_features.shape
    self.label_log_prob_ratios = np.log(label_counts / n_samples)
    
    '''
    feature_counts creates matrix for each label of size (num_labels, num_features)

    Use np.zeros to go ahead and create correctly sized matrix with 0 values for efficient memory
    '''
    feature_counts = np.zeros((len(num_labels), n_features))
    
    for idx, label in enumerate(num_labels):
        # Sum of features (word counts) for all samples in class `c`
        feature_counts[idx, :] = train_features[train_labels == label].sum(axis=0)
    
    
    # Add in Laplace smoothing (alpha) then get the smoothed feature count totals of word count per label
    smoothed_feature_counts = feature_counts + self.alpha
    smoothed_class_totals = smoothed_feature_counts.sum(axis=1).reshape(-1, 1) 
    
    # Find log of each feature given a label using the Laplace smoothed values
    self.feature_log_prob_ratios = np.log(smoothed_feature_counts) - np.log(smoothed_class_totals)
        
  def predict(self, test_features):
    '''
    test_features is the document-term matrix for the test set
    shape of test features is same as training set

    
    '''
    # Convert to dense numpy array
    test_features = np.array(test_features.todense())

    # Find the Bayesian probability using the feature log probabilities transposed and label_logs
    log_prob = np.matmul(test_features, self.feature_log_prob_ratios.T) + self.label_log_prob_ratios

    # Return the max 
    return np.argmax(log_prob, axis=1)


# END EDITING HERE - DON'T REMOVE THIS COMMENT

### DO NOT EDIT THE CODE BELOW THIS LINE

# Train
# MultinomialNB with Laplacian smoothing (alpha=1.0)
spam_filter = MultinomialNB(alpha=1.0)
spam_filter.fit(training_data, y_train)

# Evaluate
predictions = spam_filter.predict(testing_data)
from sklearn.metrics import accuracy_score, recall_score
accuracy = accuracy_score(y_test, predictions)
recall = recall_score(y_test, predictions)
print('\nPERFORMANCE')
print('Accuracy:', "{:.3f}".format(accuracy))
print('Recall:  ', "{:.3f}".format(recall))


PERFORMANCE
Accuracy: 0.989
Recall:   0.941
